#### Домашнаее задание 6


Задача 1 

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('keywords.csv')
data.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [4]:
geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}


In [5]:
def geopos(row):
    geo_loc='undefined'
    for key in geo_data.keys():
        for i in range(len(geo_data[key])):
            if geo_data[key][i] in row['keyword']:
                geo_loc=key
    return geo_loc



In [6]:
data['geo'] = data.apply(geopos, axis=1)
data['geo'].value_counts()

undefined         99253
Центр               385
Северо-Запад        268
Дальний Восток       94
Name: geo, dtype: int64

In [7]:
data.sort_values('geo', ascending=False).head()

,keyword,shows,geo
6999,москва 24,37459,Центр
19120,avto ru москва,15992,Центр
73226,фанатик спартак москва,5353,Центр
68005,гостиница москва,5651,Центр
7090,гисметео ярославль,37008,Центр


Задача 2

In [8]:
data = pd.read_csv('ml-latest-small/ratings.csv')


In [9]:
data=data.groupby('movieId').mean()

In [10]:
def movie_class(row):
    if row['rating']<=2:
        movie_class_i='Низкий рейтинг'
    if row['rating']<=4:
        movie_class_i='Средний рейтинг'
    else:
        movie_class_i='Высокий рейтинг'
    return movie_class_i

In [11]:
data['movie_class'] = data.apply(movie_class, axis=1)
data.sort_values('movie_class').head()

,userId,rating,timestamp,movie_class
movieId,,,,
163949,547.000000,5.000000,1.476419e+09,Высокий рейтинг
86817,599.000000,5.000000,1.344135e+09,Высокий рейтинг
26505,547.000000,4.500000,1.261380e+09,Высокий рейтинг
26501,547.000000,5.000000,1.249033e+09,Высокий рейтинг
4821,329.333333,4.333333,1.104783e+09,Высокий рейтинг


Задача 3 <br>
Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях.

In [16]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings_filtered = ratings.groupby('userId').count()


In [19]:
ratings_filtered = ratings_filtered[ratings_filtered['timestamp']>100].reset_index()
ratings_filtered.head()

,level_0,index,userId,movieId,rating,timestamp
0,0,0,4,204,204,204
1,1,1,8,116,116,116
2,2,2,15,1700,1700,1700
3,3,3,17,363,363,363
4,4,4,19,423,423,423


In [25]:
list_ratings=ratings_filtered['userId'].tolist()


In [26]:
film_fans_user = ratings[ ratings['userId'].isin(list_ratings)]



In [27]:

film_fans_user.groupby('userId').agg(['min', 'max'])['timestamp'].head()

,min,max
userId,,
4,949778714,949982274
8,1154389340,1154474527
15,997937239,1469330735
17,1127468587,1127476640
19,855190091,855195373


In [28]:
film_fans_users_days=film_fans_user.groupby('userId').agg(['min', 'max'])['timestamp']

In [34]:
film_fans_users_days['Lifetime']=(film_fans_users_days['max']-film_fans_users_days['min'])

In [35]:
film_fans_users_days.head()

,min,max,Lifetime
userId,,,
4,949778714,949982274,203560
8,1154389340,1154474527,85187
15,997937239,1469330735,471393496
17,1127468587,1127476640,8053
19,855190091,855195373,5282


In [42]:
from datetime import datetime
ts = int("1284101485")
film_fans_users_days['Lifetime']=pd.to_numeric(film_fans_users_days['Lifetime'])
#film_fans_users_days.info()


# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try 'ts /= 1000' in that case
film_fans_users_days['Lifetime'].apply(datetime.utcfromtimestamp(film_fans_users_days['Lifetime']).strftime('%Y-%m-%d %H:%M:%S'))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 258 entries, 4 to 671
Data columns (total 3 columns):
min         258 non-null int64
max         258 non-null int64
Lifetime    258 non-null int64
dtypes: int64(3)
memory usage: 8.1 KB
